<a href="https://colab.research.google.com/github/ycnnn/STAT542/blob/main/ver1Project1STAT542.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import basic packages

In [2]:
#import basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import xgboost as xgb

Data preprocessing

In [3]:
def preprocess(train_raw, test_raw):

    train_y_raw = train_raw['Sale_Price']
    train_raw = train_raw.drop(['PID','Sale_Price'], axis = 1)
    test_raw = test_raw.drop(['PID'], axis = 1)
    
    #first, fill all NaN values with 0
    train_raw = train_raw.fillna(0)
    test_raw = test_raw.fillna(0)


    #next, do dummy coding
    #first, pick out cat variables and numerical variables

    cat_var = []
    num_var = []

    for var in train_raw.keys().tolist():
        if train_raw[var].dtype == 'O':
            cat_var.append(var)
        else:
            num_var.append(var)


    #dummy coding train set
    #colinearity checked
    ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
    enc = ohe.fit_transform(train_raw[cat_var])
    #Converting back to a dataframe 
    train_dummy = pd.DataFrame(enc, columns=ohe.get_feature_names_out())
    test_dummy = pd.DataFrame(ohe.transform(test_raw[cat_var]), columns=ohe.get_feature_names_out())

    train_encoded = pd.concat([train_dummy,train_raw[num_var]], axis = 1)
    test_encoded = pd.concat([test_dummy,test_raw[num_var]], axis = 1)

    train_np = train_encoded.to_numpy()
    test_np = test_encoded.to_numpy()

    train_y_np = train_y_raw.to_numpy()

    return train_np, train_y_np, test_np

Model: boosting tree

In [55]:
def boost_tree_pred(train, train_y, test, tree_n = 10000):
    #define the model

    xgb_r = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = tree_n, 
                             max_depth=5, learning_rate=0.015, subsample=0.5,
                             colsample_bytree = 0.8, booster='dart',
                             reg_lambda=0.5)
   

    # Fitting the model
    xgb_r.fit(train, train_y)  
    # Predict the model
    pred = xgb_r.predict(test)
    
    return pred

In [53]:
def rmsle_calculate(predictions, dmat):
    labels = dmat.get_label()
    diffs = np.log(predictions + 1) - np.log(labels + 1)
    squared_diffs = np.square(diffs)
    avg = np.mean(squared_diffs)
    return ('RMSLE', np.sqrt(avg))

    ###############################################################################################

def rmsle(train, train_y, test,test_y,tree_n=5000):
    '''Train using native implementation of Squared Log Error.'''

    dtrain = xgb.DMatrix(train, train_y)
    dtest = xgb.DMatrix(test, test_y)

    squared_log_error = {
        'objective': 'reg:squarederror',
        'tree_method': 'hist',
        'seed': 0,
        'learning_rates': 0.05,
        'max_depth':8,
        'sub_sample':0.5,
        'booster':'dart'
    }



    bst = xgb.train(squared_log_error,
              dtrain=dtrain,
              num_boost_round=tree_n)



    pred=bst.predict(dtest)

    return pred

Import data. Data is read then pre-processed, and conveted into Numpy arrays.

In [ ]:
folds = 10
rmse = np.zeros(folds)

#load data
for ids in range(4,5):

    train_raw = pd.read_csv(f"train_{ids}.csv")
    test_raw = pd.read_csv(f"test_feature_{ids}.csv")
    test_y_raw = pd.read_csv(f"test_y_{ids}.csv")

    train, train_y, test = preprocess(train_raw, test_raw)
    test_y = test_y_raw.to_numpy().ravel()

    pred = boost_tree_pred(train, train_y, test)
    #pred = rmsle(train, train_y, test, test_y)
    
    rmse[ids] = np.sqrt(np.mean((np.log(pred) - np.log(test_y))**2))
    print(f'id:{ids}',rmse[ids])



def boost_tree_pred(train, train_y, test, tree_n = 5000):
    #define the model

    xgb_r = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = tree_n, 
                             max_depth=5, learning_rate=0.015, subsample=0.6,
                             colsample_bytree = 1)

   

    # Fitting the model
    xgb_r.fit(train, train_y)  
    # Predict the model
    pred = xgb_r.predict(test)
    
    return pred

    id:3 0.12395951345672147
id:4 0.12816665854664458